In [1]:
%matplotlib inline


# Construct Model From NeuroChem Files

This tutorial illustrates how to manually load model from `NeuroChem files`_.

    https://github.com/isayev/ASE_ANI/tree/master/ani_models


To begin with, let's first import the modules we will use:



In [2]:
import os
import torch
import torchani
import ase

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


Now let's read constants from constant file and construct AEV computer.



In [17]:
base_path  = '/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/'

In [19]:
try:
    path = os.path.dirname(os.path.realpath(__file__))
except NameError:
    path = os.getcwd()
const_file = os.path.join(base_path, 'resources/ani-1x_8x/rHCNO-5.2R_16-3.5A_a4-8.params')  # noqa: E501
print(const_file)
consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/resources/ani-1x_8x/rHCNO-5.2R_16-3.5A_a4-8.params


Now let's read self energies and construct energy shifter.



In [20]:
sae_file = os.path.join(base_path, 'resources/ani-1x_8x/sae_linfit.dat')  # noqa: E501
energy_shifter = torchani.neurochem.load_sae(sae_file)

Now let's read a whole ensemble of models.



In [21]:
model_prefix = os.path.join(base_path, 'resources/ani-1x_8x/train')  # noqa: E501
ensemble = torchani.neurochem.load_model_ensemble(consts.species, model_prefix, 8)  # noqa: E501

Or alternatively a single model.



In [22]:
model_dir = os.path.join(base_path, 'resources/ani-1x_8x/train0/networks')  # noqa: E501
model = torchani.neurochem.load_model(consts.species, model_dir)

You can create the pipeline of computing energies:
(Coordinates) -[AEVComputer]-> (AEV) -[Neural Network]->
(Raw energies) -[EnergyShifter]-> (Final energies)
From using either the ensemble or a single model:



In [24]:
nnp1 = torchani.nn.Sequential(aev_computer, ensemble, energy_shifter)
nnp2 = torchani.nn.Sequential(aev_computer, model, energy_shifter)
print(nnp1)
print(nnp2)

Sequential(
  (0): AEVComputer()
  (1): Ensemble(
    (0-7): 8 x ANIModel(
      (H): Sequential(
        (0): Linear(in_features=384, out_features=160, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=160, out_features=128, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=128, out_features=96, bias=True)
        (5): CELU(alpha=0.1)
        (6): Linear(in_features=96, out_features=1, bias=True)
      )
      (C): Sequential(
        (0): Linear(in_features=384, out_features=144, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=144, out_features=112, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=112, out_features=96, bias=True)
        (5): CELU(alpha=0.1)
        (6): Linear(in_features=96, out_features=1, bias=True)
      )
      (N): Sequential(
        (0): Linear(in_features=384, out_features=128, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=128, out_features=1

You can also create an ASE calculator using the ensemble or single model:



In [25]:
calculator1 = torchani.ase.Calculator(consts.species, nnp1)
calculator2 = torchani.ase.Calculator(consts.species, nnp2)
print(calculator1)
print(calculator1)

Now let's define a methane molecule



In [26]:
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True)
species = consts.species_to_tensor(['C', 'H', 'H', 'H', 'H']).unsqueeze(0)
methane = ase.Atoms(['C', 'H', 'H', 'H', 'H'], positions=coordinates.squeeze().detach().numpy())

Now let's compute energies using the ensemble directly:



In [27]:
energy = nnp1((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative
print('Energy:', energy.item())
print('Force:', force.squeeze())

Energy: -40.459022105724976
Force: tensor([[ 0.0306, -0.1316, -0.0527],
        [-0.1293,  0.1639, -0.0774],
        [ 0.0856, -0.0429,  0.0408],
        [ 0.0268,  0.0060,  0.0381],
        [-0.0138,  0.0046,  0.0511]])


And using the ASE interface of the ensemble:



In [28]:
methane.set_calculator(calculator1)
print('Energy:', methane.get_potential_energy() / ase.units.Hartree)
print('Force:', methane.get_forces() / ase.units.Hartree)

Energy: -40.45902210572497
Force: [[ 0.03062523 -0.13160463 -0.05265208]
 [-0.12927508  0.16388635 -0.07736803]
 [ 0.08563147 -0.04288922  0.04082094]
 [ 0.0268122   0.00601403  0.03809873]
 [-0.01379382  0.00459347  0.05110044]]


/var/folders/cq/s30xmbh57xndgmhg1zhwclsh0000gn/T/ipykernel_95412/4212972738.py:1: FutureWarning: Please use atoms.calc = calc
  methane.set_calculator(calculator1)


We can do the same thing with the single model:



In [29]:
energy = nnp2((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative
print('Energy:', energy.item())
print('Force:', force.squeeze())

methane.set_calculator(calculator2)
print('Energy:', methane.get_potential_energy() / ase.units.Hartree)
print('Force:', methane.get_forces() / ase.units.Hartree)

Energy: -40.46280035847561
Force: tensor([[ 0.0561, -0.1270, -0.0541],
        [-0.1401,  0.1552, -0.0753],
        [ 0.0753, -0.0374,  0.0395],
        [ 0.0242,  0.0024,  0.0334],
        [-0.0156,  0.0068,  0.0565]])
Energy: -40.46280035847561
Force: [[ 0.05614962 -0.12697159 -0.05413402]
 [-0.14007528  0.15523373 -0.07533836]
 [ 0.0753209  -0.03744107  0.03949668]
 [ 0.02423101  0.00235087  0.03344229]
 [-0.01562625  0.00682809  0.05653341]]


/var/folders/cq/s30xmbh57xndgmhg1zhwclsh0000gn/T/ipykernel_95412/1055837961.py:7: FutureWarning: Please use atoms.calc = calc
  methane.set_calculator(calculator2)
